# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) Base sinasc
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) número de valores missing por variável
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [5]:
# 3) faltantes das colunas especificas
sinasc[['LOCNASC','IDADEMAE','ESTCIVMAE','ESCMAE','QTDFILVIVO','GESTACAO','GRAVIDEZ','CONSULTAS','APGAR5']].isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [17]:
# 4) Removendos os missings do Apgar5 
sinasc.APGAR5.dropna()
# agora teremos 26925 linhas

0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64

In [16]:
# 5) Preenchendo os dados faltantes das colunas por 9
sinasc[['ESTCIVMAE', 'CONSULTAS']].fillna(9)

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [18]:
# 6) Substituindo os dados faltantes da QTDFILVIVO por zero
sinasc[['QTDFILVIVO']].fillna(0)

,QTDFILVIVO
0,0.0
1,1.0
2,2.0
3,0.0
4,1.0
...,...
27023,1.0
27024,0.0
27025,0.0
27026,1.0


In [20]:
# 7) verificando a melhor forma de preencher os dados faltantes das outras colunas
sinasc[['LOCNASC','IDADEMAE','ESCMAE','GESTACAO','GRAVIDEZ']]

,LOCNASC,IDADEMAE,ESCMAE,GESTACAO,GRAVIDEZ
0,1,19,8 a 11 anos,37 a 41 semanas,Única
1,1,29,8 a 11 anos,37 a 41 semanas,Única
2,1,37,8 a 11 anos,37 a 41 semanas,Única
3,1,30,12 anos ou mais,37 a 41 semanas,Única
4,1,30,8 a 11 anos,37 a 41 semanas,Única
...,...,...,...,...,...
27023,1,32,12 anos ou mais,32 a 36 semanas,Única
27024,1,19,8 a 11 anos,37 a 41 semanas,Única
27025,1,24,8 a 11 anos,37 a 41 semanas,Única
27026,1,21,8 a 11 anos,32 a 36 semanas,Única


Analisando os dados podemos observar que os dados são proximos e alguns se repetem assim preencher os valores faltantes com o valor que mais se repete pode ser bem interessante e nos ajudar na acuracia quando formos analisalos.

In [25]:
# Preenchendo valores faltantes com o valor mais frequente
most_frequent_value = sinasc[['LOCNASC','IDADEMAE','ESCMAE','GESTACAO','GRAVIDEZ']].mode()
sinasc[['LOCNASC','IDADEMAE','ESCMAE','GESTACAO','GRAVIDEZ']].fillna(most_frequent_value)

,LOCNASC,IDADEMAE,ESCMAE,GESTACAO,GRAVIDEZ
0,1,19,8 a 11 anos,37 a 41 semanas,Única
1,1,29,8 a 11 anos,37 a 41 semanas,Única
2,1,37,8 a 11 anos,37 a 41 semanas,Única
3,1,30,12 anos ou mais,37 a 41 semanas,Única
4,1,30,8 a 11 anos,37 a 41 semanas,Única
...,...,...,...,...,...
27023,1,32,12 anos ou mais,32 a 36 semanas,Única
27024,1,19,8 a 11 anos,37 a 41 semanas,Única
27025,1,24,8 a 11 anos,37 a 41 semanas,Única
27026,1,21,8 a 11 anos,32 a 36 semanas,Única


In [41]:
# 8) Transformando os dados numericos em categoricos
# Entre 8 e 10 está em uma faixa 'normal'
sinasc.loc[(sinasc['apgar5']>=8)&(sinasc['apgar5']<= 10) , 'apgar_cat'] = 'normal'
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
sinasc.loc[(sinasc['apgar5']>=6)&(sinasc['apgar5']<= 7) , 'apgar_cat'] = 'asfixia leve'
# Entre 4 e 5 significa 'asfixia moderada'.
sinasc.loc[(sinasc['apgar5']>=4)&(sinasc['apgar5']<= 5), 'apgar_cat'] = 'asfixia moderada'
# Entre 0 e 3 significa 'asfixia severa'.
sinasc.loc[(sinasc['apgar5']>=0)&(sinasc['apgar5']<= 3) , 'apgar_cat'] = 'asfixia severa'

In [44]:
# calculando a frequencia de cada categoria
sinasc.apgar_cat.value_counts()

apgar_cat
normal              26463
asfixia leve          320
nan                   103
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [32]:
# 9) Renomeando os nomes das colunas colocando as letras minúsculas com underscores entre as palavras
# Função para converter nomes de colunas para snake case
def to_snake_case(name):
    # Substituir espaços e outros caracteres por underscores, converter para minúsculas
    return name.replace(' ', '_').replace('-', '_').lower()

# Aplicar a função a todos os nomes de colunas
sinasc.columns = [to_snake_case(col) for col in sinasc.columns]

In [33]:
sinasc.head(3)

,origem,codestab,codmunnasc,locnasc,idademae,estcivmae,escmae,codocupmae,qtdfilvivo,qtdfilmort,...,kotelchuck,contador,munresstatus,munrestipo,munresnome,munresuf,munreslat,munreslon,munresalt,munresarea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
